# Retrieval API - Step 2 - Generate LLM (Gemini) Answer

## Code Initialization
Dependencies and environment initialization.

Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `.env.example` file as template.

In [1]:
import os
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [2]:
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')
GCP_MODEL = os.getenv('GCP_MODEL')

## Gemini Answer Generation

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example the retrieved articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is **Google Gemini 2.5 Flash Preview 05-20**. Response generations took a few seconds for a context between 5K to 8K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Read Chunks from [Step 1](1_get_chunks.ipynb)

In [3]:
frapi_query = u.load_dict_json('query')
chunks_list = u.load_dict_json('chunks')
frapi_prompt = frapi_query['data']['attributes']['query']['value']
display(Markdown(f"**Original Prompt:** {frapi_prompt}"))
display(Markdown(f"**Number of Chunks:** {len(chunks_list)}"))

**Original Prompt:** Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide

**Number of Chunks:** 10

### Gemini Structured Prompt

In [4]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format (don't forget the opening square bracket before the headline):
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [5]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.MRKWC00020250604el640018h",
            "source_name": "MarketWatch",
            "headline": "Car buyers drove sales to a 4-year high to beat tariffs. Now sales are running out of gas. Trade wars cause U.S. economy to seesaw",
            "publication_date": "2025-06-04",
            "content": "U.S. trade wars turbocharged sales of new cars and trucks in the spring to a four-year high, but the mad dash by consumers to avoid tariff-related pri..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250603el63002rw",
            "source_name": "Dow Jones Newswires",
            "headline": "Americans Still Hit the Car Lots in May --

### Gemini Request/Response

In [6]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION, GCP_MODEL)
display(Markdown(f"## **Gemini Model:** {GCP_MODEL}"))
display(Markdown(f"## **Prompt:** {frapi_prompt}"))
display(Markdown(f"## **Gemini Response:**"))
display(Markdown(response))

## **Gemini Model:** gemini-2.5-flash-preview-05-20

## **Prompt:** Summarise tariff decisions by the US administration over the last few months and their impact on the automotive industry worldwide

## **Gemini Response:**

Over the last few months, the US administration has made several tariff decisions impacting the automotive industry worldwide. These decisions include the doubling of steel and aluminum tariffs, the imposition of new tariffs on imported vehicles and auto components, and the negotiation of specific trade agreements.

**Key Tariff Decisions by the US Administration:**
*   **Increased Steel and Aluminum Tariffs:** The Trump administration has doubled tariffs on steel and aluminum to 50% [1, 3, 6].
*   **Auto Import Tariffs:** A 25% import tariff on cars has been in place [7, 10].
*   **Tariffs on Indian Auto Products:** The US adopted a 25% ad valorem tariff increase on imports of passenger vehicles, light trucks, and certain automobile parts originating from India, effective May 3, 2025, for an unlimited duration [4, 6, 9].
*   **UK Trade Agreement:** A recent trade agreement with the US includes a 100,000 quota for a 10% tariff on car imports from the UK [5].
*   **Call for Domestic Parts Manufacturing:** President Trump has expressed a desire for parts manufacturing to return to the US [7].

**Impact on the Automotive Industry Worldwide:**

**On the US Automotive Market and Consumers:**
*   **Sales Volatility:** Consumers rushed to purchase new cars and trucks in March and April 2025 to avoid tariff-related price increases, driving sales to a four-year high (17.8 million annual rate in March). However, this surge was followed by a nearly 10% drop in May, with sales returning to a more typical pace (15.6 million annual rate) [1, 2].
*   **Increased Costs and Prices:** Rising costs of imported goods and domestic steel have compelled some carmakers to raise prices [1]. The average price for a new vehicle in May 2025 was $45,462 [2]. Doubling steel and aluminum tariffs could add approximately 5% to cumulative tariff costs, potentially exceeding $7,000 per car [3].
*   **Consumer Confidence:** Concerns about inflation and the impact of higher tariffs have contributed to low consumer confidence [2].
*   **Inventory Challenges:** Dealers have begun running out of vehicles acquired before the tariffs took effect [1].
*   **Uncertainty and Outlook:** Economists predict slower and more uneven growth for the US economy due to trade fights [1]. Ford, for instance, suspended its regular forecast of future sales and profits due to this uncertainty [1]. Auto sales in March are anticipated to be the high point of the year, with a slowdown expected before a rebound next year [1].

**On Global Automotive Manufacturers and Trade Relations:**
*   **Manufacturing Costs:** The increased metals tariffs are expected to raise manufacturing costs for carmakers in the US and complicate efforts to build and sell more vehicles [1, 3].
*   **Company-Specific Impacts:**
    *   **Toyota:** Reported that US tariffs added $1.25 billion to its costs in April and May 2025, as it imports about 45% of the vehicles it sells in the US [2].
    *   **Mazda:** Experienced a 19% plunge in sales in May, largely due to its reliance on imports [2].
    *   **Korean Automakers (Hyundai Motor, Kia, GM Korea):** Face a "double whammy" of declining exports and domestic slump, with the 25% US auto tariff expected to cause significant earnings damage [10]. Abolishing this tariff is a top priority for South Korea in its trade negotiations with the US [10].
    *   **Ford, General Motors, Stellantis:** Their stock prices have been negatively affected by tariff concerns [3, 7].
*   **Supply Chain Challenges:** The push for domestic parts manufacturing poses a long-term challenge, as it is difficult to produce all components within the US, particularly lower-value parts or those no longer manufactured there (e.g., LED displays) [7]. Low-cost parts are crucial for keeping new-car prices affordable [7].
*   **International Trade Disputes:**
    *   **India:** Has sought consultations with the US at the World Trade Organization (WTO) over the 25% tariffs on passenger vehicles, light trucks, and auto components, arguing these are unnotified safeguard measures. India has reserved the right to impose retaliatory duties and seeks compensation [4, 6, 9]. India's auto component exports to the US were estimated at $6.8 billion in FY24 [6].
    *   **Mexico:** Mexican companies are increasingly utilizing the Mexico-United States-Canada Agreement (T-MEC) to ensure tariff-free access to the US market, as the US has imposed significant tariffs on other trading partners [8]. The T-MEC requires 75% regional content for automobiles to qualify for preferential treatment [8]. Without T-MEC, average tariffs for Mexican automotive exports to the US would be substantially higher (8.1%) [8].
    *   **United Kingdom:** While a recent trade agreement with the US provides a 100,000 quota for a 10% tariff on car imports, which Jaguar Land Rover considers the "best deal" under the circumstances, the UK steel industry faces significant revenue losses compared to Europe due to price divergence caused by US tariffs [5].

**Cited Articles:**
- [1] [Car buyers drove sales to a 4-year high to beat tariffs. Now sales are running out of gas. Trade wars cause U.S. economy to seesaw - MarketWatch - 2025-06-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.MRKWC00020250604el640018h)
- [2] [Americans Still Hit the Car Lots in May -- WSJ - Dow Jones Newswires - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250603el63002rw)
- [3] [特朗普计划上调钢铝关税的举动导致汽车股普遍走低 - 道琼斯中文通讯 (简体) - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.RTCNWS0020250602el620008b)
- [4] [India seeks WTO consultations with US over safeguard measures on auto components - Press Trust of India - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.PRTRIN0020250603el6300c1g)
- [5] [The recent trade agreement with the US "is the very best deal that the UK could... - Press Association National Newswire - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.PRESSA0020250603el63006mx)
- [6] [India seeks consultations with US at WTO on auto tariffs - BusinessLine Online - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.BSNLNO0020250603el630010g)
- [7] [Ford, GM Stocks Are Slipping. Investors Want a Brake on Tariffs. -- Barrons.com - Dow Jones Newswires - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.DJDN000020250602el62002gf)
- [8] [Automotive industry, the largest exporter within T-MEC - CE NAFTA 2.0-USMCA - 2025-06-02](https://dj.factiva.com/article?id=drn:archive.newsarticle.NAFTA00020250602el620002z)
- [9] [India invokes WTO mechanism to address US safeguard duties on auto components - The Times of India - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.TOI0000020250603el630008l)
- [10] [Carmakers face double whammy of export fall, domestic slump amid leadership vacuum - Korea Times - 2025-06-03](https://dj.factiva.com/article?id=drn:archive.newsarticle.KORTIM0020250603el630000i)